# Assignment 6 - Implementing k-means with Text Data

In this assignment you will implement the k-means algorithm. Unlike past homework assignments, you are not using a pre-implemented sklearn class; you will be implementing the code for k-means yourself.  The assignment uses the `numpy` library for manipulating the data arrays. See the following tutorial for more information on `numpy`.

* **[NumPy Tutorial](https://numpy.org/doc/stable/user/quickstart.html)**

We will be focusing on the specific setting of clustering text documents, but your k-means implementation will be general to any setting. When properly executed, clustering uncovers valuable insights from a set of unlabeled documents.

In this assignment, you will:

* Cluster Wikipedia documents using k-means
* Explore the role of random initialization on the quality of the clustering
* Explore how results differ after changing the number of clusters
* Evaluate clustering, both quantitatively and qualitatively

Fill in the cells provided marked `TODO` with code to answer the questions. 

> Copyright ©2023 Emily Fox and Hunter Schafer.  All rights reserved.  Permission is hereby granted to students registered for University of Washington CSE/STAT 416 for use solely during Spring Quarter 2024 for purposes of the course.  No other use, copying, distribution, or modification is permitted without prior written consent. Copyrights for third-party components of this work must be honored.  Instructors interested in reusing these course materials should contact the author.

---

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

# Load data, Extract features

In [62]:
### SKIP
wiki = pd.read_csv('people_wiki.csv')
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Mauno_J%C3%A4rvel...,Mauno J%C3%A4rvel%C3%A4,mauno jrvel born 25 november 1949 in kaustinen...
1,<http://dbpedia.org/resource/David_W._Jourdan>,David W. Jourdan,david walter jourdan born december 5 1954 is a...
2,<http://dbpedia.org/resource/Patrick_Roach>,Patrick Roach,patrick roach born march 4 1969 is a canadian ...
3,<http://dbpedia.org/resource/Louis_Sauer>,Louis Sauer,louis lou sauer aka louis edward sauer born 19...
4,<http://dbpedia.org/resource/Marty_Keough>,Marty Keough,richard martin keough born april 14 1934 in oa...


In [63]:
### edTest(test_load_data) ###

To work with text data, we must first convert the documents into numerical features. Let's extract TF-IDF features for each article.

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.95)  # ignore words with very high doc frequency
tf_idf = vectorizer.fit_transform(wiki['text'])
words = vectorizer.get_feature_names_out()

Since most documents don't contain every word, many of the TF-IDF entries will be 0. Representing the TF-IDF matrix as a `numpy` matrix will require a lot of unnecessary storage to keep track of all those 0. `scikit-learn` instead represents it as a SciPy "sparse matrix" that only represents the non-zero entries of a matrix to save space. Externally, you treat it just like a numpy `matrix` but it takes up less storage.

The above matrix contains a TF-IDF score for each of the 5907 pages in the data set and each of the 112801 unique words. You can treat it like any matrix of data but internally it is space-efficient since it knows not to store the zero word counts.

# Normalize all vectors

Euclidean distance can be a poor metric of similarity between documents, since it is very sensitive to the length of an article. As an example, we want long and short articles about baseball players to be treated as similar, but Euclidean distance would treat them as very different due to the length. For a better assessment of similarity, we should disregard the length information and use length-agnostic metrics, such as cosine distance. Unfortunately, the k-means algorithm doesn't work with non-Euclidean distance metrics.

We take an alternative route to remove length information: we normalize all vectors to be unit length. It turns out that Euclidean distance closely mimics cosine distance when all vectors are unit length. In particular, the squared Euclidean distance between any two vectors of length one is directly proportional to their cosine distance.

---

### Optional: Justification
This section has some optional background material as to why normalizing makes sense here. You can skip down to the next line break if you don't want to read this.

We can prove this as follows. Let $\mathbf{x}$ and $\mathbf{y}$ be normalized vectors, i.e. unit vectors, so that $\|\mathbf{x}\|=\|\mathbf{y}\|=1$. Write the squared Euclidean distance as the dot product of $(\mathbf{x} - \mathbf{y})$ to itself:

$$\begin{align*}
\|\mathbf{x} - \mathbf{y}\|_2^2 &= (\mathbf{x} - \mathbf{y})^T(\mathbf{x} - \mathbf{y}) & \text{(def of L2 norm)}\\
                              &= (\mathbf{x}^T \mathbf{x}) - 2(\mathbf{x}^T \mathbf{y}) + (\mathbf{y}^T \mathbf{y}) & \text{(FOIL expression)}\\
                              &= \|\mathbf{x}\|_2^2 - 2(\mathbf{x}^T \mathbf{y}) + \|\mathbf{y}\|_2^2 & \text{(def of L2 norm)}\\
                              &= 2 - 2(\mathbf{x}^T \mathbf{y}) & \text{($\mathbf{x}$ and $\mathbf{y}$ are length 1)}\\
                              &= 2(1 - (\mathbf{x}^T \mathbf{y}))\\
                              &= 2\left(1 - \frac{\mathbf{x}^T \mathbf{y}}{\|\mathbf{x}\|_2\|\mathbf{y}\|_2}\right) & \text{(Dividing by 1 doesn't change value)}\\
                              &= 2 \cdot cosine\_distance(\mathbf{x}, \mathbf{y})
\end{align*}$$

This tells us that two **unit vectors** that are close in Euclidean distance are also close in cosine distance. Thus, the k-means algorithm (which naturally uses Euclidean distances) on normalized vectors will produce the same results as clustering using cosine distance as a distance metric.

*End optional section*.

---
We import the [`normalize()` function](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html) from scikit-learn to normalize all vectors to unit length.

In [65]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)
tf_idf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 993501 stored elements and shape (5907, 112801)>

# Implement k-means

The bulk of this assignment will be implementing k-means. We will tackle it in parts to make it manageable.

First, we choose an initial set of centroids. A common practice is to choose randomly from the data points.

**Note:** We specify a seed here, so that everyone gets the same answer. In practice, we highly recommend to use different seeds every time (for instance, by using the current timestamp).

In [66]:
def get_initial_centroids(data, k, seed=None):
    """
    Randomly choose k data points as initial centroids
    """
    if seed is not None: # useful for obtaining consistent results
        np.random.seed(seed)
        
    n = data.shape[0] # number of data points
        
    # Pick K indices from range [0, n) without replacement.
    rand_indices = np.random.choice(n, k)
    
    # Keep centroids as dense (not sparse) matrix format, as many entries 
    # will be nonzero due to averaging. As long as at least one document 
    # in a cluster contains a word, it will carry a nonzero weight in the 
    # TF-IDF vector of the centroid.
    centroids = data[rand_indices,:].toarray()
    
    return centroids

### k-means Algorithm
After initialization, the k-means algorithm iterates between the following two steps:
1. Assign each data point to the closest centroid. $$z_i \gets \mathrm{argmin}_{j\in[k]} \|\mathbf{\mu}_j - \mathbf{x}_i\|_2^2$$
2. Revise centroids to be the mean of the assigned data points. $$\mu^{(j)} \gets \frac{\sum_{i=1}^n \mathbb{1}\{z_i = j\}x_i}{\sum_{i=1}^n \mathbb{1}\{z_i = j\}}$$

In pseudocode, we iteratively do the following:
```python
cluster_assignment = assign_clusters(data, centroids)
centroids = revise_centroids(data, k, cluster_assignment)
```

## Assigning clusters

How do we implement Step 1 of the main k-means loop above? First we import `pairwise_distances` function from scikit-learn, which calculates Euclidean distances between rows of given arrays. See [this documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html) for more information.

For the sake of demonstration, let's look at documents 100 through 102 as query documents and compute the distances between each of these documents and every other document in the corpus. In the k-means algorithm, we will have to compute pairwise distances between the set of centroids and the set of documents.

In [67]:
from sklearn.metrics import pairwise_distances

# Get the TF-IDF vectors for documents 100 through 102.
queries = tf_idf[100:102,:]

# Compute pairwise distances from every data point to each query vector.
dist = pairwise_distances(tf_idf, queries, metric='euclidean')
print(dist)

[[1.39996239 1.39958932]
 [1.40386156 1.39754968]
 [1.38421176 1.39682604]
 ...
 [1.40562888 1.39024794]
 [1.39673862 1.38306708]
 [1.40872806 1.40250208]]


More formally, `dist[i,j]` is assigned the distance between the `i`th row of `X` (i.e., `X[i,:]`) and the `j`th row of `Y` (i.e., `Y[j,:]`).

### 🔍 **Question 1** Computing Distances

To test your understanding of how this code works, in the cell below write code that does the following tasks

* Initializes 3 centroids that are the first 3 rows of `tf_idf`
* Compute the distances between all the points in `tf_idf` and those 3 centroids. The result should be a matrix with shape `(5907, 3)`. Store this in a variable called `distances`.
* Use `distances` to find the distance between the row of `tf_idf` with index 430 to the second centroid (index 1). Store this value in a variable called `dist`.

In [68]:
### edTest(test_q1_computing_distances) ###

# TODO Fill out this cell
centroids = tf_idf[0:3,:].toarray()

distances = pairwise_distances(tf_idf, centroids, metric='euclidean')
dist = distances[430, 1]

print("\ncentroids")
print(centroids)

print("\ndistance")
print(distances)

print("\ndist")
print(dist)


centroids
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

distance
[[0.00000000e+00 1.40383275e+00 1.39333920e+00]
 [1.40383275e+00 2.10734243e-08 1.39553028e+00]
 [1.39333920e+00 1.39553028e+00 2.35608046e-08]
 ...
 [1.39157805e+00 1.39141281e+00 1.38192050e+00]
 [1.39962336e+00 1.38790661e+00 1.38090390e+00]
 [1.40691565e+00 1.40462283e+00 1.40161262e+00]]

dist
1.403199223315177


### 🔍 **Question 2** Find closest centroid

Now that you have computed the pairwise distances between all datapoints in `tf_idf` and the 3 centroids, your task is to find the closest centroid for each datapoint and store it in the variable `closest_cluster`. To do so, for each datapoint find the the centroid with the minimum distance to the datapoint. Fittingly, NumPy provides an `argmin` function. See [this documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.argmin.html) for details. 

`closest_cluster` should be a 1D array whose $i^{th}$ entry contains the index of the centroid that is the closest to the $i^{th}$ datapoint. Note that it would be **unreasonably slow** run this code if you use a for/while loop; therefore, you **must use [argmin](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.argmin.html)**. (See documentation for examples)

*Hint:* the resulting array should be as long as the number of data points. Think carefully about which axis you want to take the argmin on to get the index of the centroid with the minimum distance to that datapoint.

In [69]:
### edTest(test_q2_closest_centroid) ###

# TODO Fill out this cell
closest_cluster = ...

### 🔍 **Question 3** Assign Clusters

Now that we have completed components of the Step 1 code, let's put it together in a single function that takes a dataset and centroids and assigns each row to the closest centroid. We are ready to fill in the blanks in the aforementioned function `assign_clusters(data, centroids)`. 

In [70]:
### edTest(test_q3_assign_clusters) ###

# TODO Complete this function
def assign_clusters(data, centroids):
    """
    Parameters:  
      - data      - is an np.array of float values with n rows and d columns.  
      - centroids - is an np.array of float values with k rows and dcolumns.

    Returns  
      -  A np.array of length n where the ith index represents which centroid 
         data[i] was assigned to. The assignments range between the values 0, ..., k-1.
    """
    # TODO get the distance between data and centroids
    
    # TODO get the closest centroid for each datapoint

    # TODO return an array that contains the closest centroid index for each datapoint
    return ...
    

## Revising clusters

### Numpy Tutorial
Let's turn to Step 2 of the k-means algorithm, where we compute the new centroids given the current cluster assignments. 

SciPy and NumPy arrays allow for filtering via Boolean masks. For instance, we filter all data points that are assigned to cluster 0 by writing

```python
data[cluster_assignment == 0, :]
```

To develop intuition about filtering, let's look at a small example consisting of 3 data points and 2 clusters.

In [71]:
data = np.array([[1., 2., 0.],
                 [0., 0., 0.],
                 [2., 2., 0.]])
centroids = np.array([[0.5, 0.5, 0.],
                      [0., -0.5, 0.]])

Let's assign these data points to the closest centroid using the function you wrote before.

In [72]:
cluster_assignment = assign_clusters(data, centroids)
cluster_assignment

Ellipsis

The expression `cluster_assignment == 1` gives a list of Booleans that says whether each data point is assigned to cluster 1 or not:

In [73]:
cluster_assignment == 1

False

Likewise for cluster 0:

In [74]:
cluster_assignment == 0

False

When indexing into a numpy array, instead of indices, we can also put in the list of Booleans to pick and choose rows. Only the rows that correspond to a `True` entry will be retained.

First, let's look at the values of the data points that were assigned to cluster 1:

In [75]:
data[cluster_assignment == 1]

array([], shape=(0, 3, 3), dtype=float64)

This makes sense since the vector `[0 0 0]` is closer to the centroid `[0 -0.5 0]` than to the centroid `[0.5 0.5 0]`.

Now let's look at the data points assigned to cluster 0:

In [76]:
data[cluster_assignment == 0]

array([], shape=(0, 3, 3), dtype=float64)

Again, this makes sense since these values are each closer to the centroid `[0.5 0.5 0]` than to `[0 -0.5 0]`.

Given all the data points in a cluster, it only remains to compute the mean. Use [np.mean()](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.mean.html). By default, the function averages all elements in a 2D array. To compute row-wise or column-wise means, add the `axis` argument. See the linked documentation for details. 

In the cell below, we first find all the rows that were assigned cluster 0 and then take the average of those vectors to find the new cluster 0 centroid. Notice that the result will be an np.array with 3 elements because that's the dimensionality of the vectors.

In [77]:
data[cluster_assignment==0].mean(axis=0)

C:\Users\achar\AppData\Local\Temp\ipykernel_28960\1342920855.py:1: RuntimeWarning: Mean of empty slice.
  data[cluster_assignment==0].mean(axis=0)
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

### 🔍 **Question 4** Revise Centroids


Now we are ready to fill in the blanks in the function `revise_centroids(data, k, cluster_assignment)`. In the cell below, complete the `...` sections to compute the new centroids given the current cluster assignment.

In [78]:
### edTest(test_q4_revise_centroids) ###

# TODO fill in this function
def revise_centroids(data, k, cluster_assignment):
    """
    Parameters:  
      - data               - is an np.array of float values with n rows and d columns.
      - k                  - number of centroids
      - cluster_assignment - np.array of length n where the ith index represents which 
                             centroid data[i] was assigned to. The assignments range between the values 0, ..., k-1.

    Returns  
      -  A np.array with k rows and d columns for the new centroids.
    """
    new_centroids = []
    for i in range(k):
        # TODO Select all data points that belong to cluster i. Fill in the ... portion
        member_data_points = ...
        
        # TODO Compute the mean of the member data points. Fill in the ... portion
        centroid = ...
        
        # Convert numpy.matrix type to numpy.ndarray type
        centroid = centroid.A1
        new_centroids.append(centroid)
        
    new_centroids = np.array(new_centroids)
    return new_centroids

## Assessing convergence

How can we tell if the k-means algorithm is converging? We can look at the cluster assignments and see if they stabilize over time. In fact, we'll be running the algorithm until the cluster assignments stop changing at all. To be extra safe, and to assess the clustering performance, we'll be looking at an additional criteria: the sum of all squared distances between data points and centroids (called the "heterogeneity objective" in lecture). This is defined as
$$
J(\mathcal{Z},\mu) = \sum_{j=1}^{k} \sum_{i=1}^n \mathbb{1}\{z_i =j\}\|x_i - \mu^{(j)}\|^2_2.
$$
The smaller the distances, the more homogeneous the clusters are. In other words, we'd like to have "tight" clusters.

In [79]:
def compute_heterogeneity(data, k, centroids, cluster_assignment):
    """
    Computes the heterogeneity metric of the data using the given centroids and cluster assignments.
    """
    heterogeneity = 0.0
    for i in range(k):
        
        # Select all data points that belong to cluster i. Fill in the blank (RHS only)
        member_data_points = data[cluster_assignment == i, :]
        
        if member_data_points.shape[0] > 0: # check if i-th cluster is non-empty
            # Compute distances from centroid to data point
            distances = pairwise_distances(member_data_points, [centroids[i]], metric='euclidean')
            squared_distances = distances ** 2
            heterogeneity += np.sum(squared_distances)
        
    return heterogeneity

    

Let's compute the cluster heterogeneity for the 2-cluster example we've been considering based on our current cluster assignments and centroids.

In [80]:
print(compute_heterogeneity(data, 2, centroids, cluster_assignment))
cluster_assignment.shape

0.0


AttributeError: 'ellipsis' object has no attribute 'shape'

### 🔍 **Question 5** Combining it into a single function

Once the two k-means steps have been implemented, as well as our heterogeneity metric we wish to monitor, it is only a matter of putting these functions together to write a k-means algorithm that

* Repeatedly performs Steps 1 and 2
* Tracks convergence metrics on each iteration
* Stops if either no assignment changed or we reach a certain number of iterations.

Now we are ready to fill in the blanks the function `kmeans(data, k, initial_centroids, maxiter, record_heterogeneity=None, verbose=False)`. In the cell below, complete the `...` sections to meet the specification of the function.

In [ ]:
### edTest(test_q5_kmeans) ###

# TODO Fill in the blanks
def kmeans(data, k, initial_centroids, max_iter, record_heterogeneity=None, verbose=False):
    """
    This function runs k-means on given data and initial set of centroids.
    
    Parameters:  
      - data                 - is an np.array of float values of length N.
      - k                    - number of centroids
      - initial_centroids    - is an np.array of float values of length k.
      - max_iter             - maximum number of iterations to run the algorithm
      - record_heterogeneity - if provided an empty list, it will compute the heterogeneity 
                               at each iteration and append it to the list. 
                               Defaults to None and won't record heterogeneity.
      - verbose              - set to True to display progress. Defaults to False and won't 
                               display progress.

    Returns  
      - centroids - A np.array of length k for the centroids upon termination of the algorithm.
      - cluster_assignment - A np.array of length n where the ith index represents which 
                             centroid data[i] was assigned to. The assignments range between the 
                             values 0, ..., k-1 upon termination of the algorithm.
    """
    centroids = initial_centroids[:]
    prev_cluster_assignment = None
    
    for itr in range(max_iter):  
        # Print itereation number
        if verbose:
            print(itr)
        
        # TODO 1. Make assign each datapoint to the nearest centroid
        cluster_assignment = ...
            
        # TODO 2. Compute a new centroid for each of the k clusters, by averaging all 
        # data points assigned to that cluster.
        centroids = ...
            
        # Check for convergence: if none of the assignments changed, stop
        if prev_cluster_assignment is not None and \
          (prev_cluster_assignment == cluster_assignment).all():
            break
        
        # Print number of new assignments 
        if prev_cluster_assignment is not None:
            num_changed = sum(abs(prev_cluster_assignment - cluster_assignment))
            if verbose:
                print(f'    {num_changed:5d} elements changed their cluster assignment.')  
        
        # Record heterogeneity convergence metric
        if record_heterogeneity is not None:
            # TODO compute the heterogeneity of the cluster
            score = ...
            record_heterogeneity.append(score)
        
        prev_cluster_assignment = cluster_assignment[:]
        
    return centroids, cluster_assignment

## Plotting convergence metric

We can use the above function to plot the convergence metric across iterations.

In [ ]:
def plot_heterogeneity(heterogeneity, k):
    """
    Plots how the heterogeneity changes as the number of iterations increases.
    """
    plt.figure(figsize=(7,4))
    plt.plot(heterogeneity, linewidth=4)
    plt.xlabel('# Iterations')
    plt.ylabel('Heterogeneity')
    plt.title(f'Heterogeneity of clustering over time, K={k}')
    plt.rcParams.update({'font.size': 16})
    plt.tight_layout()

Let's consider running k-means with $k=3$ clusters for a maximum of 400 iterations, recording cluster heterogeneity at every step.  Then, let's plot the heterogeneity over iterations using the plotting function above. We include a seed to ensure everyone gets the same results.

In [ ]:
k = 3
heterogeneity = []
initial_centroids = get_initial_centroids(tf_idf, k, seed=0)
centroids, cluster_assignment = kmeans(tf_idf, k, initial_centroids, max_iter=400,
                                       record_heterogeneity=heterogeneity, verbose=True)
plot_heterogeneity(heterogeneity, k)

### 🔍 **Question 6** Largest cluster

Using the output of k-means from the last cell, write code to compute which final cluster contains the most data points. Store your result as the index of that cluster (an `int`) in a variable named `largest_cluster`, 
representing which cluster in `centroids` has the most datapoints assigned to it. You may want to use [np.bincount](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html).


In [ ]:
### edTest(test_q6_largest_cluster) ###

# TODO Find largest cluster from example above
largest_cluster = ...


# Beware of Local Minima

One weakness of k-means is that it tends to get stuck in a local minimum based on its starting position. To see this, let us run k-means multiple times, with different initial centroids created using different random seeds.

**Note:** Again, in practice, you should set different seeds for every run. We give you a list of seeds for this assignment so that everyone gets the same answer.

This may take a minute or two to run.

In [ ]:
%%time
# ^ Magic command to time how long it takes for this cell to run!
# You can see how long it took with the output that says "Wall time"

k = 10
heterogeneity = {}
for seed in [0, 20000, 40000, 60000, 80000, 100000, 120000]:
    initial_centroids = get_initial_centroids(tf_idf, k, seed)
    centroids, cluster_assignment = kmeans(tf_idf, k, initial_centroids, max_iter=800,
                                           record_heterogeneity=None, verbose=False)
    # To save time, compute heterogeneity only once in the end
    heterogeneity[seed] = compute_heterogeneity(tf_idf, k, centroids, cluster_assignment)
    print(f'seed={seed:06d}, heterogeneity={heterogeneity[seed]:.5f}')


Notice the variation in heterogeneity for different initializations. This indicates that k-means runs may have not converged or they got stuck at a local minimum.

# k-means++
One effective way to counter this tendency is to use **k-means++** to provide a smart initialization. This method tries to spread out the initial set of centroids so that they are not too close together. It is known to improve the quality of local optima and lower average runtime, but is a bit slower to start since it needs to do more computation to place centroids.

In [ ]:
def k_means_plus_plus_initialization(data, k, seed=None):
    """
    Use k-means++ to initialize a good set of centroids
    """
    if seed is not None: # useful for obtaining consistent results
        np.random.seed(seed)
        
    centroids = np.zeros((k, data.shape[1]))
    
    # Randomly choose the first centroid.
    # Since we have no prior knowledge, choose uniformly at random
    idx = np.random.randint(data.shape[0])
    centroids[0] = data[idx,:].toarray()
    
    # Compute distances from the first centroid chosen to all the other data points
    distances = pairwise_distances(data, centroids[0:1], metric='euclidean').flatten()
    
    for i in range(1, k):
        # Choose the next centroid randomly, so that the probability for each data point to be chosen
        # is directly proportional to its squared distance from the nearest centroid.
        # Roughtly speaking, a new centroid should be as far as from ohter centroids as possible.
        idx = np.random.choice(data.shape[0], 1, p=distances/sum(distances))
        centroids[i] = data[idx,:].toarray()
        
        # Now compute distances from the centroids to all data points
        distances = np.min(pairwise_distances(data, centroids[0:i+1], metric='euclidean'),axis=1)
    
    return centroids

Let's now rerun k-means with 10 clusters using the same set of seeds, but always using k-means++ to initialize the algorithm.

This may take several minutes to run.

In [ ]:
%%time

k = 10
heterogeneity_smart = {}
for seed in [0, 20000, 40000, 60000, 80000, 100000, 120000]:
    initial_centroids = k_means_plus_plus_initialization(tf_idf, k, seed)
    centroids, cluster_assignment = kmeans(tf_idf, k, initial_centroids, max_iter=400,
                                           record_heterogeneity=None, verbose=False)
    # To save time, compute heterogeneity only once in the end
    heterogeneity_smart[seed] = compute_heterogeneity(tf_idf, k, centroids, cluster_assignment)
    print(f'seed={seed:06d}, heterogeneity={heterogeneity_smart[seed]:.5f}')

Let's compare the set of cluster heterogeneities we got from our 7 restarts of k-means using random initialization compared to the 7 restarts of k-means using k-means++ as a smart initialization.

The following code produces a [box plot](http://matplotlib.org/api/pyplot_api.html) for each of these methods, indicating the spread of heterogeneity values produced by each method.

In [ ]:
plt.figure(figsize=(10,5))
plt.boxplot([list(heterogeneity.values()), list(heterogeneity_smart.values())], vert=False)

plt.yticks([1, 2], ['k-means', 'k-means++'])
plt.xlabel('Heterogeneity')
plt.title('Heterogeneity distribution of k-means vs k-means++')

plt.rcParams.update({'font.size': 16})
plt.tight_layout()

A few things to notice from the box plot:
* Random initialization results in a worse clustering than k-means++ on average.
* The best result of k-means++ is better than the best result of random initialization.

## 🔍 **Question 7** kmeans_multiple_runs
**In general, you should run k-means at least a few times with different initializations and then return the run resulting in the lowest heterogeneity.** Let us write a function that runs k-means multiple times and picks the best run that minimizes heterogeneity.

Now we are ready to fill in the blanks the function `kmeans_multiple_runs(data, k, max_iter, verbose=False)`. 



In [ ]:
### edTest(test_q7_kmeans_multiple_runs) ###

# TODO Fill in the ...
def kmeans_multiple_runs(data, k, max_iter, seeds, verbose=False):
    """
    Runs kmeans multiple times 
    
    Parameters:  
      - data     - is an np.array of float values of length n.
      - k        - number of centroids
      - max_iter - maximum number of iterations to run the algorithm
      - seeds    - Either number of seeds to try (generated randomly) or a list of seed values
      - verbose  - set to True to display progress. Defaults to False and won't display progress.
    
    Returns  
      - final_centroids          - A np.array of length k for the centroids upon 
                                   termination of the algorithm.
      - final_cluster_assignment - A np.array of length n where the ith index represents which 
                                   centroid data[i] was assigned to. The assignments range between 
                                   the values 0, ..., k-1 upon termination of the algorithm.
    """    
    min_heterogeneity_achieved = float('inf')
    final_centroids = None
    final_cluster_assignment = None
    if type(seeds) == int:
        seeds = np.random.randint(low=0, high=10000, size=seeds)
    
    num_runs = len(seeds)
    
    for seed in seeds:
        # TODO Use k-means++ initialization: Fill in the blank
        # Set record_heterogeneity=None because we will compute that once at the end.
        initial_centroids = ...

        # TODO Run k-means: Fill in the blank 
        centroids, cluster_assignment = ...
        
        # TODO To save time, compute heterogeneity only once in the end
        seed_heterogeneity = ...
        
        if verbose:
            print(f'seed={seed:06d}, heterogeneity={seed_heterogeneity:.5f}')
        
        # if current measurement of heterogeneity is lower than previously seen,
        # update the minimum record of heterogeneity.
        if seed_heterogeneity < min_heterogeneity_achieved:
            min_heterogeneity_achieved = seed_heterogeneity
            final_centroids = centroids
            final_cluster_assignment = cluster_assignment
    
    # Return the centroids and cluster assignments that minimize heterogeneity.
    return final_centroids, final_cluster_assignment

## How to choose K

Since we are measuring the tightness of the clusters, a higher value of K reduces the possible heterogeneity metric by definition.  For example, if we have N data points and set K=N clusters, then we could have 0 cluster heterogeneity by setting the N centroids equal to the values of the N data points. (Note: Not all runs for larger K will result in lower heterogeneity than a single run with smaller K, but heterogeneity will always stay the same of decrease as you increase $k$.)  Let's explore this general trend for ourselves by performing the following analysis.


This code block will take some times to complete. It will try 5 values of `k` and for each `k`, will try 3 different seeds. The cell will print its progress to help you know how far it has made. When `k` is larger, it will take longer to run (why might that be?)!

In [ ]:
%%time

def plot_k_vs_heterogeneity(k_values, heterogeneity_values):
    """
    Given list of k-values and their heterogeneities, will make a plot
    showing how heterogeneity varies with k.
    """
    plt.figure(figsize=(7,4))
    plt.plot(k_values, heterogeneity_values, linewidth=4)
    plt.xlabel('K')
    plt.ylabel('Heterogeneity')
    plt.title('K vs. Heterogeneity')
    plt.rcParams.update({'font.size': 16})
    plt.tight_layout()

all_centroids = {}
all_cluster_assignment = {}
heterogeneity_values = []
seeds = [20000, 40000, 80000]
k_list = [2, 10, 25, 50, 100]

for k in k_list:
    print(f'Running k = {k}')
    heterogeneity = []
    all_centroids[k], all_cluster_assignment[k] = kmeans_multiple_runs(tf_idf, k, max_iter=400,
                                                                       seeds=seeds, verbose=True)
    score = compute_heterogeneity(tf_idf, k, all_centroids[k], all_cluster_assignment[k])
    heterogeneity_values.append(score)

plot_k_vs_heterogeneity(k_list, heterogeneity_values)

# Inspecting clusters of documents
Let's start visualizing some clustering results to see if we think the clustering makes sense.  We can use such visualizations to help us assess whether we have set K too large or too small for a given application.  Following the theme of this course, we will judge whether the clustering makes sense in the context of document analysis.

What are we looking for in a good clustering of documents?
* Documents in the same cluster should be similar.
* Documents from different clusters should be less similar.

So a bad clustering exhibits either of two symptoms:
* Documents in a cluster have mixed content.
* Documents with similar content are divided up and put into different clusters.

To help visualize the clustering, we do the following:
* Fetch nearest neighbors of each centroid from the set of documents assigned to that cluster. We will consider these documents as being representative of the cluster.
* Print titles and first sentences of those nearest neighbors.
* Print top 5 words that have highest tf-idf weights in each centroid.

In [ ]:
def visualize_document_clusters(wiki, tf_idf, centroids, cluster_assignment, k, words, 
                                display_docs=5):
    """
    Given a set of clustered documents, prints information about the centroids including
       - The title and starting sentence of the closest 5 points to each centroid
       - The five words that are contained in the clusters documents with the highest TF-IDF.
    
    Parameters:  
      - wiki: original dataframe
      - tf_idf: data matrix containing TF-IDF vectors for each document
      - centroids: A np.array of length k that contains the centroids for the clustering
      - cluster_assignments: A np.array of length N that has the cluster assignments for each row
      - k: What value of k is used
      - words: List of words in the corpus (should match tf_idf)
      - display_odcs: How many documents to show for each cluster (default 5)
    """
    print('=' * 90)

    # Visualize each cluster c
    for c in range(k):
        # Cluster heading
        print(f'Cluster {c}  ({(cluster_assignment == c).sum()} docs)'),
        # Print top 5 words with largest TF-IDF weights in the cluster
        idx = centroids[c].argsort()[::-1]
        for i in range(5): # Print each word along with the TF-IDF weight
            print(f'{words[idx[i]]}:{centroids[c,idx[i]]:.3f}', end=' '),
        print()
        
        if display_docs > 0:
            print()
            # Compute distances from the centroid to all data points in the cluster,
            # and compute nearest neighbors of the centroids within the cluster.
            distances = pairwise_distances(tf_idf, centroids[c].reshape(1, -1), metric='euclidean').flatten()
            distances[cluster_assignment!=c] = float('inf') # remove non-members from consideration
            nearest_neighbors = distances.argsort()
            # For the nearest neighbors, print the title as well as first 180 characters of text.
            # Wrap the text at 80-character mark.
            for i in range(display_docs):
                text = ' '.join(wiki.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
                print(f'* {wiki.iloc[nearest_neighbors[i]]["name"]:50s} {distances[nearest_neighbors[i]]:.5f}')
                print(f'  {text[:90]}')
                if len(text) > 90:
                    print(f'  {text[90:180]}')
                print()
        print('=' * 90)

Let us first look at the 2 cluster case (K=2).

In [ ]:
k = 2
visualize_document_clusters(wiki, tf_idf, all_centroids[k], all_cluster_assignment[k], k, words)

Both clusters have mixed content, although it appears that cluster 0 has women and cluster 1 has men.

It would be better if we sub-divided into more categories. So let us use more clusters. How about `K=10`?

In [ ]:
k = 10
visualize_document_clusters(wiki, tf_idf, all_centroids[k], all_cluster_assignment[k], k, words)

We no longer have the clear split between men and women. Cluters 0 and 2 appear to be still mixed, but others are quite consistent in content.
* Cluster 0: notable women
* Cluster 1: baseball players
* Cluster 2: researchers, professors
* Cluster 3: football(soccer)
* Cluster 4: musicians, singers, song writers
* Cluster 5: golfers
* Cluster 6: painters, scultpers, artists
* Cluster 7: orchestral musicians, conductors
* Cluster 8: politicians, political personel
* Cluster 9: film directors

Clusters are now more pure (only having one category), but some are qualitatively "bigger" than others. For instance, the category of scholars is more general than the category of film directors. Increasing the number of clusters may split larger clusters. Another way to look at the size of cluster is to count the number of articles in each cluster.

In [ ]:
np.bincount(all_cluster_assignment[10])

There appears to be at least some connection between the topical consistency of a cluster and the number of its member data points.

Let us visualize the case for K=25. For the sake of brevity, we do not print the content of documents. Let's print out the top words with highest TF-IDF weights, to understand what type of documents tend to be in the clusters.


In [ ]:
k = 25
visualize_document_clusters(wiki, tf_idf, all_centroids[k], all_cluster_assignment[k], k,
                            words, display_docs=0) # turn off text for brevity

Looking at the representative examples and top words, we classify each cluster as follows. Notice the bolded items, which indicate the appearance of a new theme that did not appear to be represented with $k=10$.
* Cluster 0: **British labor party**
* Cluster 1: **Bishops**
* Cluster 2: **danish CEOs**
* Cluster 3: baseball
* Cluster 4: politicials
* Cluster 5: **psychology researchers**
* Cluster 6: **medical researchers**
* Cluster 7: **republican politicians**
* Cluster 8: football(soccer)
* Cluster 9: **prime ministers**
* Cluster 10: golfers
* Cluster 11: coaches
* Cluster 12: **lawers**
* Cluster 13: researchers, professors
* Cluster 14: writers
* Cluster 15: artists, museaum workers
* Cluster 16: film directors
* Cluster 17: musicians
* Cluster 18: **airforce commanders**
* Cluster 19: orchestral musicians
* Cluster 20: *unclear*
* Cluster 21: *unclear*
* Cluster 22: *unclear*
* Cluster 23: politicians
* Cluster 24: **hockey players**

Indeed, increasing K achieved the desired effect of breaking up large clusters.  Depending on the application, this may or may not be preferable to the K=10 analysis.

Let's take it to the extreme and set K=100. We have a suspicion that this value is too large. Let us look at the top words from each cluster:

In [ ]:
k = 100
visualize_document_clusters(wiki, tf_idf, all_centroids[k], all_cluster_assignment[k], k,
                            words, display_docs=0)

**A high value of K encourages pure clusters, but we cannot keep increasing K. For large enough K, related documents end up going to different clusters.**

That said, the result for K=100 is not entirely bad. After all, it gives us separate clusters for such categories as Scotland, Brazil, LGBT, computer science and the Mormon Church. If we set K somewhere between 25 and 100, we should be able to avoid breaking up clusters while discovering new ones.

Also, we should ask ourselves how much **granularity** we want in our clustering. If we wanted a rough sketch of Wikipedia, we don't want too detailed clusters. On the other hand, having many clusters can be valuable when we are zooming into a certain part of Wikipedia.

**There is no golden rule for choosing K. It all depends on the particular application and domain we are in.**

Another heuristic people use that does not rely on so much visualization, which can be hard in many applications (including here!) is as follows.  Track heterogeneity versus K and look for the **"elbow"** of the curve where the heterogeneity decrease rapidly before this value of K, but then only gradually for larger values of K.  This naturally trades off between trying to minimize heterogeneity, but reduce number of clusters.  In the heterogeneity versus K plot made above, we did not yet really see a flattening out of the heterogeneity, which might indicate that indeed K=100 is "reasonable" and we only diminishing returns for larger values of K (which are even harder to visualize using the methods we attempted above.)

### 🔍 **Question 8** Small clusters

Another sign of too large K is having lots of small clusters. Look at the distribution of cluster sizes (by number of member data points). When doing k-means with $k=100$, how many of the clusters have fewer than 44 articles (i.e. 0.004% of the dataset)?

Save your result in a variable called `num_small_clusters`


In [ ]:
### edTest(test_q8_small_clusters) ###

# TODO count clusters with fewer than 44 articles
num_small_clusters = ...

Keep in mind though that tiny clusters aren't necessarily bad. A tiny cluster of documents that really look like each others is definitely preferable to a medium-sized cluster of documents with mixed content. However, having too few articles in a cluster may lead us to question if that cluster is really worth separating from the others.